# Librairies

In [1]:
import project_functions as pf
import pandas as pd

# Portfolio Analysis


Importons les données :

In [2]:
# récupérons les données des prix sauvegardés dans le fichier prices.csv
prices = pd.read_csv('datas/prices.csv', sep=',')
prices = prices.set_index('Date')

## Analyse des returns

Nous ne prenons uniquement les assets avec des returns anualisés positifs.

In [3]:
# Calculons les rendements journaliers
daily_returns = pf.daily_asset_return(prices)

# Calculons les rendements annuels
annualy_returns = pf.annualy_asset_return(daily_returns)

# On ne concerne uniquement les index avec rendements annuels positifs
positive_returns = annualy_returns[annualy_returns > 0]

# On récupère les tickers des index avec rendements positifs
tickers = positive_returns.index.to_list()

Actualisons le dataframe prices avec uniquement les tickers avec un rendement positif

In [4]:
prices = prices[tickers]

## Critères ESG

Nous allons créer notre portefeuille d'actions en se basant sur les critères ESG. Comme vu en cours cela s'appelle un ESG filter :
* exclude x% of firms with the worts ESG score
* keep firms with the best ESG momentum
* take a specific KPI

In [13]:
# récupérons les données liées aux critères ESG sauvegardés dans le fichier esg_scores.csv
esg = pd.read_csv('datas/esg_scores.csv', sep=',')

# On ne conserve que les tickers avec des scores ESG différents de NaN
esg = esg.dropna()

In [14]:
# On conserve les tickers avec un score environnemental supérieur à la médiane
esg = esg.sort_values(by=['Environment Score'])
tickers_env = esg.iloc[n//2:]
tickers_env

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
64,64,WHA.AS,85.0,67.0,51.0,69.0
34,34,INGA.AS,86.0,68.0,89.0,78.0
58,58,URW.AS,88.0,95.0,88.0,90.0
22,22,DSM.AS,89.0,79.0,86.0,84.0
52,52,SHELL.AS,92.0,93.0,95.0,93.0
8,8,AMUND.AS,93.0,78.0,42.0,63.0


In [15]:
# On conserve les tickers avec un score social supérieur à la médiane
esg = esg.sort_values(by=['Social Score'])
tickers_soc = esg.iloc[n//2:]
tickers_soc

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
9,9,ARCAD.AS,78.0,90.0,73.0,81.0
24,24,FUR.AS,57.0,90.0,84.0,77.0
38,38,MT.AS,81.0,91.0,75.0,83.0
52,52,SHELL.AS,92.0,93.0,95.0,93.0
45,45,PHIA.AS,81.0,94.0,84.0,88.0
58,58,URW.AS,88.0,95.0,88.0,90.0


In [20]:
# Concaténation des dataframes en supprimant les doublons
tickers_esg = pd.concat([tickers_env, tickers_soc]).drop_duplicates()
tickers_esg

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
64,64,WHA.AS,85.0,67.0,51.0,69.0
34,34,INGA.AS,86.0,68.0,89.0,78.0
58,58,URW.AS,88.0,95.0,88.0,90.0
22,22,DSM.AS,89.0,79.0,86.0,84.0
52,52,SHELL.AS,92.0,93.0,95.0,93.0
8,8,AMUND.AS,93.0,78.0,42.0,63.0
9,9,ARCAD.AS,78.0,90.0,73.0,81.0
24,24,FUR.AS,57.0,90.0,84.0,77.0
38,38,MT.AS,81.0,91.0,75.0,83.0
45,45,PHIA.AS,81.0,94.0,84.0,88.0


In [24]:
# Récupérons la liste des tickers
list_esg = tickers_esg['Ticker Yahoo'].to_list()
print(f'Nombre de tickers ESG sélectionnés : {len(list_esg)}')

Nombre de tickers ESG sélectionnés : 10


## Analyse des Volatilités

Nous calculons la volatilites des actions avec la fonction asset_vol.

In [25]:
# Calculons la volatilité annuelle de chaque actif et trions les valeurs par ordre décroissant
total_vol = pf.asset_vol(prices).sort_values(ascending=True)
total_vol.head()

ROOD.AS      0.748127
AGN.AS      12.640179
EAS2P.AS    14.934748
CTAC.AS     16.252868
IEX.AS      17.617157
dtype: float64

Récupérons les 10 tickers avec la plus faibles volatilité. Et supprimons les doublons s'il y en a avec ceux déjà sélectionnés selon les critères ESG.

In [29]:
list_vol = total_vol.index.to_list()[:10]

## Concaténons les deux listes

In [34]:
tickers = list_esg + list_vol

# Supprimons les doublons
tickers = list(dict.fromkeys(tickers))

## Enregistrement des tickers

Nous devons enregistrer les tickers sélectionnés dans le fichier tickers.txt

In [35]:
pf.tickers_storage(tickers, 'datas/tickers.txt')